In [1]:
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Codes/Shared Codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/Results'
feature_selection_results = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/feature_selection_results'
sys.path.insert(0,code_path)
from oselm import set_use_know
from SharedFunctions import main,generate_oselm_models

In [5]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))

In [6]:
data_name = ['kddcup99','ISCX2012','CSE-CIC2018','CICIDS2017','7recurrentDrift', 'blip', 'incrementalDrift','7gradualDrift', '7suddenDrift']

In [7]:
# # use_know = True
# # set_use_know(use_know)
# # path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
# # os.makedirs(path, exist_ok=True)
# # for f_name,d_name in zip(filenames,data_name):
# #   DFS_results_path = os.path.join(feature_selection_results,d_name)
# #   main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
# #       feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,DFS_results_path=DFS_results_path)

In [8]:
DATA_NUMBER=0
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.9955099422706863, 'precision': 1.0, 'recall': 0.991494532199271, 'f1-score': 0.9957291031116534, 'auc': 0.9957472660996355}


  0%|          | 0/7832 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.9639240506329114, 'precision': 0.9379391100702577, 'recall': 0.9950310559006211, 'f1-score': 0.9656419529837251, 'auc': 0.9633219795632137}


  0%|          | 0/7892 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.9879670677644079, 'precision': 0.9864864864864865, 'recall': 0.9901356350184957, 'f1-score': 0.9883076923076923, 'auc': 0.9879063591759144}


  0%|          | 0/7790 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.9922978177150192, 'precision': 0.9897172236503856, 'recall': 0.9948320413436692, 'f1-score': 0.9922680412371134, 'auc': 0.992313979855508}


In [9]:
DATA_NUMBER=1
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : ISCX2012.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.6339341205717837, 'precision': 0.5692913385826772, 'recall': 0.9450980392156862, 'f1-score': 0.7105651105651106, 'auc': 0.6484968869064214}


  0%|          | 0/8018 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.7066167290886392, 'precision': 0.993920972644377, 'recall': 0.41132075471698115, 'f1-score': 0.5818505338078293, 'auc': 0.7044212199855042}


  0%|          | 0/8138 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.6363636363636364, 'precision': 0.9872340425531915, 'recall': 0.28258221680876977, 'f1-score': 0.4393939393939394, 'auc': 0.6394323723449052}


  0%|          | 0/8122 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.5076923076923077, 'precision': 0.90625, 'recall': 0.035108958837772396, 'f1-score': 0.06759906759906759, 'auc': 0.515677132735532}


In [10]:
DATA_NUMBER=2
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CSE-CIC2018.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 0.5}


  0%|          | 0/8000 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'auc': 1.0}


  0%|          | 0/15962 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.9996868149076104, 'precision': 0.9993630573248408, 'recall': 1.0, 'f1-score': 0.9996814272061166, 'auc': 0.999692118226601}


  0%|          | 0/15986 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.9996873045653534, 'precision': 1.0, 'recall': 0.9993626513702996, 'f1-score': 0.999681224099458, 'auc': 0.9996813256851498}


In [11]:
DATA_NUMBER=3
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : CICIDS2017.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.9823885525591635, 'precision': 0.9707207207207207, 'recall': 0.9930875576036866, 'f1-score': 0.9817767653758542, 'auc': 0.982845148664857}


  0%|          | 0/8974 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.8905723905723906, 'precision': 0.8221198156682028, 'recall': 0.9977628635346756, 'f1-score': 0.9014653865588682, 'auc': 0.8902102605961666}


  0%|          | 0/8874 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.8529577464788732, 'precision': 0.8257173219978746, 'recall': 0.8890160183066361, 'f1-score': 0.8561983471074381, 'auc': 0.8534980202520971}


  0%|          | 0/8898 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.7443820224719101, 'precision': 0.8722689075630252, 'recall': 0.5779510022271714, 'f1-score': 0.695244474212994, 'auc': 0.7458916008868285}


In [12]:
DATA_NUMBER=4
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7recurrentDrift.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.6445820433436532, 'precision': 0.6555555555555556, 'recall': 0.6428571428571429, 'f1-score': 0.6491442542787287, 'auc': 0.6446224877783813}


  0%|          | 0/8120 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.6234567901234568, 'precision': 0.6170212765957447, 'recall': 0.6727716727716728, 'f1-score': 0.6436915887850468, 'auc': 0.6229026903184207}


  0%|          | 0/8176 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.5971882640586798, 'precision': 0.585958595859586, 'recall': 0.7658823529411765, 'f1-score': 0.6639469658337582, 'auc': 0.5903203113306391}


  0%|          | 0/8082 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.6784168212739641, 'precision': 0.7084507042253522, 'recall': 0.616421568627451, 'f1-score': 0.6592398427260814, 'auc': 0.6789973011676582}


In [13]:
DATA_NUMBER=5
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : blip.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.49890470974808326, 'precision': 0.5431654676258992, 'recall': 0.16080937167199147, 'f1-score': 0.24815119145439604, 'auc': 0.508815057876582}


  0%|          | 0/8994 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.4915806626833243, 'precision': 0.49472990777338605, 'recall': 0.816304347826087, 'f1-score': 0.6160787530762921, 'auc': 0.4917569513288958}


  0%|          | 0/9138 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.4907002188183807, 'precision': 0.48887293610911703, 'recall': 0.7566666666666667, 'f1-score': 0.5939816833842129, 'auc': 0.4947126436781609}


  0%|          | 0/9030 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.4988925802879291, 'precision': 0.49592944369063774, 'recall': 0.8185890257558791, 'f1-score': 0.6176594845796367, 'auc': 0.5023941842908639}


In [14]:
DATA_NUMBER=6
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : incrementalDrift.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.7014276846679082, 'precision': 0.7309236947791165, 'recall': 0.6610169491525424, 'f1-score': 0.6942148760330579, 'auc': 0.7024829968692649}


  0%|          | 0/8072 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.5735475896168108, 'precision': 0.5620915032679739, 'recall': 0.7314702308626975, 'f1-score': 0.6356916578669483, 'auc': 0.5707665619722293}


  0%|          | 0/8008 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.6504369538077404, 'precision': 0.617258883248731, 'recall': 0.7686472819216182, 'f1-score': 0.6846846846846847, 'auc': 0.6518945410841136}


  0%|          | 0/8092 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.46201358863495984, 'precision': 0.4481641468682505, 'recall': 0.535483870967742, 'f1-score': 0.48794826572604344, 'auc': 0.4650168170004587}


In [15]:
DATA_NUMBER=7
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7gradualDrift.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.6242632612966601, 'precision': 0.5829447035309794, 'recall': 0.8629191321499013, 'f1-score': 0.6958250497017894, 'auc': 0.6251973351551855}


  0%|          | 0/10100 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.5839091806515301, 'precision': 0.5460704607046071, 'recall': 0.8232890704800817, 'f1-score': 0.6566191446028513, 'auc': 0.5916827396335461}


  0%|          | 0/10178 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.619351669941061, 'precision': 0.5911528150134048, 'recall': 0.8424068767908309, 'f1-score': 0.6947617172115006, 'auc': 0.6128111229252435}


  0%|          | 0/10160 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.6304133858267716, 'precision': 0.6094325718496684, 'recall': 0.7891221374045801, 'f1-score': 0.6877338877338878, 'auc': 0.6252521256128591}


In [16]:
DATA_NUMBER=8
ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_DFS')
os.makedirs(path, exist_ok=True)
f_name,d_name = filenames[DATA_NUMBER],data_name[DATA_NUMBER]
DFS_results_path = os.path.join(feature_selection_results,d_name)
main(f_name, generate_oselm_models, apply_model_replacement=True,transfer_learning=True,
     feature_selection=["with_out_feature_evolving",'DFS_feature_selection'], result_save_path=path,
     DFS_results_path=DFS_results_path,ChunkNumber=ChunkNumber)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : 7suddenDrift.csv ======================


0it [00:00, ?it/s]

DFS
1 {'accuracy': 0.6246290801186943, 'precision': 0.5782575173477256, 'recall': 0.7796257796257796, 'f1-score': 0.6640106241699868, 'auc': 0.6317940218883615}


  0%|          | 0/10062 [00:00<?, ?it/s]

...

DFS
8 {'accuracy': 0.6106194690265486, 'precision': 0.5808671713695802, 'recall': 0.8218111002921129, 'f1-score': 0.6806451612903227, 'auc': 0.6085222333635342}


  0%|          | 0/10216 [00:00<?, ?it/s]

DFS
9 {'accuracy': 0.6394324853228963, 'precision': 0.5981035740335522, 'recall': 0.8151093439363817, 'f1-score': 0.689945309213294, 'auc': 0.6421404137793661}


  0%|          | 0/10234 [00:00<?, ?it/s]

DFS
10 {'accuracy': 0.5818270639960919, 'precision': 0.5501706484641639, 'recall': 0.8035892323030908, 'f1-score': 0.6531604538087521, 'auc': 0.586181589331622}
